In [ ]:
#### Runs RMSD ####
### DONE

import os
import MDAnalysis as mda
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import nglview as nv
import ipywidgets as w
import ipykernel
from collections import defaultdict
from MDAnalysis.analysis import align, rms, diffusionmap
from MDAnalysisTests.datafiles import PSF, DCD
import MDAnalysis.transformations as trans
%matplotlib inline

### NOTE: Need to define how to deal with bad beginning of trajectories before continuing
### SOLVE: Propose that the first X frames (find a suitable number of ns, not a whole much % of total) were for equilibration and discard them --- Currently 75 frames, accounting for 1,5 ns out of 30 ns.

systems = ['6q2b', '5h3r']
conditions = {'6q2b': ['wild', 'm84', 'm63'], '5h3r': ['wild', 'm94', 'm73']}
mutations = {'6q2b': ['wild', 'k84r', 'r63t'], '5h3r': ['wild', 'r94k', 'r73t']}
rmsd_data = {}

def all_plots_exist(systems, conditions, reps):
    for system in systems:
        for condition in conditions[system]:
            for rep in reps:
                fig_path_same_system = f'rmsd_plot_{system}_all_conditions_rep{rep}.png'
                if not os.path.exists(fig_path_same_system):
                    return False

                equivalent_system = '5h3r' if system == '6q2b' else '6q2b'
                if 'm' in condition:
                    number = int(condition[1:])
                    equivalent_number = number + 10 if system == '6q2b' else number - 10
                    equivalent_condition = f'm{equivalent_number}'
                else:
                    equivalent_condition = condition

                if system <= equivalent_system:
                    fig_path_equivalent_system = f'rmsd_plot_{system}_{condition}_vs_{equivalent_system}_{equivalent_condition}_rep{rep}.png'
                    if not os.path.exists(fig_path_equivalent_system):
                        return False
    return True

reps = range(1, 4)
if all_plots_exist(systems, conditions, reps):
    print("All plots exist. Skipping RMSD calculation.")
else:
    print("Starting RMSD calculation for each system, condition, and replicate...")
    for system in systems:
        for condition in conditions[system]:
            mutation = mutations[system][conditions[system].index(condition)]
            for rep in range(1, 4):
                print(f"Processing {system}, {condition}, replicate {rep}...")
                dcd_file = f'{system}/analises_gerais/{mutation}/{system}_production_{condition}_reduced_rep{rep}.dcd'
                aligned_dcd_file = f'rmsfit_{system}_production_{condition}_reduced_rep{rep}.dcd'
                prmtop_file = f'{system}/analises_gerais/{mutation}/{system}_system_{condition}.top'

                if os.path.exists(aligned_dcd_file):
                    print(f"Using pre-aligned trajectory from {aligned_dcd_file}")
                    u = mda.Universe(prmtop_file, aligned_dcd_file, dt=200)
                    ref = mda.Universe(prmtop_file, dcd_file, dt=200)
                    protein = u.select_atoms('protein')
                    not_protein = u.select_atoms('not protein')
                    dna = u.select_atoms('nucleic')

                    transforms = [trans.unwrap(protein),
                                trans.center_in_box(protein, wrap=True),
                                trans.wrap(not_protein)]

                    u.trajectory.add_transformations(*transforms)

                    u_nucleic = u.select_atoms('nucleic')
                    ref_nucleic = ref.select_atoms('nucleic')

                    print("Calculating RMSD...")
                    R_rmsd = rms.RMSD(u_nucleic, ref_nucleic, select='nucleic', ref_frame=0)
                    R_rmsd.run(start=75, stop=1500, step=1)

                    rmsd_data[(system, condition, rep)] = R_rmsd

                    del u, protein, not_protein, dna, transforms, u_nucleic, ref_nucleic, R_rmsd

                else:
                    print(f"Aligning trajectory from {dcd_file}")
                    u = mda.Universe(prmtop_file, dcd_file, dt=200)
                    ref = mda.Universe(prmtop_file, dcd_file, dt=200)

                    protein = u.select_atoms('protein')
                    not_protein = u.select_atoms('not protein')
                    dna = u.select_atoms('nucleic')

                    transforms = [trans.unwrap(protein),
                                trans.center_in_box(protein, wrap=True),
                                trans.wrap(not_protein)]

                    u.trajectory.add_transformations(*transforms)

                    print("Running alignment...")
                    aligner = align.AlignTraj(u, ref, select='protein', in_memory=False).run(start=75, stop=1500, step=1)
                    u_nucleic = u.select_atoms('nucleic')
                    ref_nucleic = ref.select_atoms('nucleic')

                    print("Calculating RMSD...")
                    R_rmsd = rms.RMSD(u_nucleic, ref_nucleic, select='nucleic', ref_frame=0)
                    R_rmsd.run(start=75, stop=1500, step=1)

                    rmsd_data[(system, condition, rep)] = R_rmsd

                    del u, protein, not_protein, dna, transforms, aligner, u_nucleic, ref_nucleic, R_rmsd
print("Finished RMSD calculation.")

def plot_equivalent_systems(rmsd_data, rep):
    for system in ['6q2b', '5h3r']:
        equivalent_system = '5h3r' if system == '6q2b' else '6q2b'
        for condition in conditions[system]:
            if 'm' in condition:
                number = int(condition[1:])
                equivalent_number = number + 10 if system == '6q2b' else number - 10
                equivalent_condition = f'm{equivalent_number}'
            else:
                equivalent_condition = condition

            # Avoid creating duplicated graphs
            if system > equivalent_system:
                continue

            fig_path = f'rmsd_plot_{system}_{condition}_vs_{equivalent_system}_{equivalent_condition}_rep{rep}.png'
            if os.path.exists(fig_path):
                print(f"Skipping plot for {system}_{condition}_vs_{equivalent_system}_{equivalent_condition}_rep{rep} because it already exists.")
                continue

            try:
                rmsd_df_system = pd.DataFrame(rmsd_data[(system, condition, rep)].results.rmsd, columns=['Frame', 'Time (ns)', f'{system}_{condition}'])
                rmsd_df_equivalent = pd.DataFrame(rmsd_data[(equivalent_system, equivalent_condition, rep)].results.rmsd, columns=['Frame', 'Time (ns)', f'{equivalent_system}_{equivalent_condition}'])
                rmsd_df = pd.merge(rmsd_df_system, rmsd_df_equivalent, on=['Frame', 'Time (ns)'])

                # Check if 'Frame' column is empty or all values are the same
                if rmsd_df['Frame'].empty or rmsd_df['Frame'].nunique() == 1:
                    print(f"Skipping plot for {system}_{condition}_vs_{equivalent_system}_{equivalent_condition}_rep{rep} due to insufficient 'Frame' data.")
                    continue

                ax = rmsd_df.plot(x='Frame', y=[f'{system}_{condition}', f'{equivalent_system}_{equivalent_condition}'], kind='line')
                ax.set_ylabel(r'RMSD ($\AA$)')
                fig = ax.get_figure()
                fig.savefig(f'rmsd_plot_{system}_{condition}_vs_{equivalent_system}_{equivalent_condition}_rep{rep}.png')
                plt.close(fig)
            except Exception as e:
                print(f"Failed to plot {system}_{condition}_vs_{equivalent_system}_{equivalent_condition}_rep{rep}. Error: {str(e)}")

def plot_same_system(rmsd_data, rep):
    for system in ['6q2b', '5h3r']:
        fig_path = f'rmsd_plot_{system}_all_conditions_rep{rep}.png'
        if os.path.exists(fig_path):
            print(f"Skipping plot for {system}_all_conditions_rep{rep} because it already exists.")
            continue
        rmsd_df = pd.DataFrame()
        for condition in conditions[system]:
            rmsd_df_temp = pd.DataFrame(rmsd_data[(system, condition, rep)].results.rmsd, columns=['Frame', 'Time (ns)', f'{system}_{condition}'])
            if rmsd_df.empty:
                rmsd_df = rmsd_df_temp
            else:
                rmsd_df = pd.merge(rmsd_df, rmsd_df_temp, on=['Frame', 'Time (ns)'])
        ax = rmsd_df.plot(x='Frame', y=[f'{system}_{condition}' for condition in conditions[system]], kind='line')
        ax.set_ylabel(r'RMSD ($\AA$)')
        fig = ax.get_figure()
        fig.savefig(f'rmsd_plot_{system}_all_conditions_rep{rep}.png')
        plt.close(fig)

print("Starting to plot equivalent systems...")
for rep in range(1, 4):
    plot_equivalent_systems(rmsd_data, rep)
print("Finished plotting equivalent systems.")

print("Starting to plot same system, different conditions...")
for rep in range(1, 4):
    plot_same_system(rmsd_data, rep)
print("Finished plotting same system, different conditions.")

In [ ]:
#### Runs RMSF and RoG, triplicates all at once for each system ####
#### PENDING


import os
import MDAnalysis as mda
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import nglview as nv
import ipywidgets as w
import ipykernel
from collections import defaultdict
from MDAnalysis.analysis import align, rms, diffusionmap
from MDAnalysisTests.datafiles import PSF, DCD
import MDAnalysis.transformations as trans
%matplotlib inline

### NOTE: Need to define how to deal with bad beginning of trajectories before continuing
### SOLVE: Propose that the first X frames (find a suitable number of ns, not a whole much % of total) were for equilibration and discard them --- Currently 75 frames, accounting for 1,5 ns out of 30 ns.

systems = ['6q2b', '5h3r']
conditions = {'6q2b': ['wild', 'm84', 'm63'], '5h3r': ['wild', 'm94', 'm73']}
mutations = {'6q2b': ['wild', 'k84r', 'r63t'], '5h3r': ['wild', 'r94k', 'r73t']}
rmsd_data = {}

print("Starting 2D-RMSD, RMSF and RoG calculation for each system, condition, and replicate...")
for system in systems:
    for condition in conditions[system]:
        mutation = mutations[system][conditions[system].index(condition)]
        for rep in range(1, 4):
            print(f"Processing {system}, {condition}, replicate {rep}...")
            dcd_file = f'{system}/analises_gerais/{mutation}/{system}_production_{condition}_reduced_rep{rep}.dcd'
            aligned_dcd_file = f'rmsfit_{system}_production_{condition}_reduced_rep{rep}.dcd'
            prmtop_file = f'{system}/analises_gerais/{mutation}/{system}_system_{condition}.top'

            if os.path.exists(aligned_dcd_file):
                print(f"Using pre-aligned trajectory from {aligned_dcd_file}")
                u = mda.Universe(prmtop_file, aligned_dcd_file, dt=200)
                ref = mda.Universe(prmtop_file, dcd_file, dt=200)
                protein = u.select_atoms('protein')
                not_protein = u.select_atoms('not protein')
                dna = u.select_atoms('nucleic')

                transforms = [trans.unwrap(protein),
                            trans.center_in_box(protein, wrap=True),
                            trans.wrap(not_protein)]

                u.trajectory.add_transformations(*transforms)

                u_nucleic = u.select_atoms('nucleic')
                ref_nucleic = ref.select_atoms('nucleic')

                print("Calculating RMSD...")
                R_rmsd = rms.RMSD(u_nucleic, ref_nucleic, select='nucleic', ref_frame=0)
                R_rmsd.run(start=75, stop=1500, step=1)

                rmsd_data[(system, condition, rep)] = R_rmsd

                del u, protein, not_protein, dna, transforms, u_nucleic, ref_nucleic, R_rmsd

### RMSF Calculation ###
### TODO: Gotta pass all 3 trajectories at once to generate a single RMSF plot
### Maybe we'll need to iterate in a different manner than in other analysis, but I think not
### Just run the RMSF universe generation and the rest when inside a key (wild, R63T, K84R...) for the first time

### TODO: Split the selection in half, only calculating for one of the monomers

# average = align.AverageStructure(u, u, select='protein and backbone',
#                                  ref_frame=0).run()
# ref_rmsf = average.results.universe

# aligner = align.AlignTraj(u, ref_rmsf,
#                           select='protein and backbone',
#                           in_memory=True).run()

# u_rmsf_backbone = u.select_atoms('protein and backbone')
# R_rmsf = rms.RMSF(u_rmsf_backbone).run()


# fig, ax = plt.subplots()
# ax.plot(u_rmsf_backbone.resids, R_rmsf.results.rmsf)
# ax.set_xlabel('Residue number')
# ax.set_ylabel('RMSF ($\AA$)')
# ax.legend()

### TODO: Write conditional to color the residues 63 and 84 if 6q2b and 73 and 94 if 5h3r

# # ax.axvspan(122, 159, zorder=0, alpha=0.2, color='orange', label='LID')
# # ax.axvspan(30, 59, zorder=0, alpha=0.2, color='green', label='NMP')

# fig.savefig(f'rmsf_plot_{protein_plotting}_{key}_{repl_no}.png')
# plt.close(fig)

### Radius of Gyration Calculation ###
# rog = []
# time = []
# protein = u.select_atoms('protein')

# protein

# for ts in u.trajectory:
#     time.append(u.trajectory.time)
#     rog.append(protein.radius_of_gyration())

# rog_df = pd.DataFrame(rog, columns=['Radius of gyration (A)'], index=time)
# rog_df.index.name = 'Time (ps)'
# rog_df.plot(title='Radius of gyration')
# plt.savefig(f'radius_of_gyration.png')


def all_plots_exist(systems, conditions, reps):
    for system in systems:
        for condition in conditions[system]:
            for rep in reps:
                fig_path_same_system = f'rmsd_plot_{system}_all_conditions_rep{rep}.png'
                if not os.path.exists(fig_path_same_system):
                    return False

                equivalent_system = '5h3r' if system == '6q2b' else '6q2b'
                if 'm' in condition:
                    number = int(condition[1:])
                    equivalent_number = number + 10 if system == '6q2b' else number - 10
                    equivalent_condition = f'm{equivalent_number}'
                else:
                    equivalent_condition = condition

                if system <= equivalent_system:
                    fig_path_equivalent_system = f'rmsd_plot_{system}_{condition}_vs_{equivalent_system}_{equivalent_condition}_rep{rep}.png'
                    if not os.path.exists(fig_path_equivalent_system):
                        return False
    return True

reps = range(1, 4)
if all_plots_exist(systems, conditions, reps):
    print("All plots exist. Skipping RMSD calculation.")
else:
    print("Starting RMSD calculation for each system, condition, and replicate...")
    for system in systems:
        for condition in conditions[system]:
            mutation = mutations[system][conditions[system].index(condition)]
            for rep in range(1, 4):
                print(f"Processing {system}, {condition}, replicate {rep}...")
                dcd_file = f'{system}/analises_gerais/{mutation}/{system}_production_{condition}_reduced_rep{rep}.dcd'
                aligned_dcd_file = f'rmsfit_{system}_production_{condition}_reduced_rep{rep}.dcd'
                prmtop_file = f'{system}/analises_gerais/{mutation}/{system}_system_{condition}.top'

                if os.path.exists(aligned_dcd_file):
                    print(f"Using pre-aligned trajectory from {aligned_dcd_file}")
                    u = mda.Universe(prmtop_file, aligned_dcd_file, dt=200)
                    ref = mda.Universe(prmtop_file, dcd_file, dt=200)
                    protein = u.select_atoms('protein')
                    not_protein = u.select_atoms('not protein')
                    dna = u.select_atoms('nucleic')

                    transforms = [trans.unwrap(protein),
                                trans.center_in_box(protein, wrap=True),
                                trans.wrap(not_protein)]

                    u.trajectory.add_transformations(*transforms)

                    u_nucleic = u.select_atoms('nucleic')
                    ref_nucleic = ref.select_atoms('nucleic')

                    print("Calculating RMSD...")
                    R_rmsd = rms.RMSD(u_nucleic, ref_nucleic, select='nucleic', ref_frame=0)
                    R_rmsd.run(start=75, stop=1500, step=1)

                    rmsd_data[(system, condition, rep)] = R_rmsd

                    del u, protein, not_protein, dna, transforms, u_nucleic, ref_nucleic, R_rmsd

                else:
                    print(f"Aligning trajectory from {dcd_file}")
                    u = mda.Universe(prmtop_file, dcd_file, dt=200)
                    ref = mda.Universe(prmtop_file, dcd_file, dt=200)

                    protein = u.select_atoms('protein')
                    not_protein = u.select_atoms('not protein')
                    dna = u.select_atoms('nucleic')

                    transforms = [trans.unwrap(protein),
                                trans.center_in_box(protein, wrap=True),
                                trans.wrap(not_protein)]

                    u.trajectory.add_transformations(*transforms)

                    print("Running alignment...")
                    aligner = align.AlignTraj(u, ref, select='protein', in_memory=False).run(start=75, stop=1500, step=1)
                    u_nucleic = u.select_atoms('nucleic')
                    ref_nucleic = ref.select_atoms('nucleic')

                    print("Calculating RMSD...")
                    R_rmsd = rms.RMSD(u_nucleic, ref_nucleic, select='nucleic', ref_frame=0)
                    R_rmsd.run(start=75, stop=1500, step=1)

                    rmsd_data[(system, condition, rep)] = R_rmsd

                    del u, protein, not_protein, dna, transforms, aligner, u_nucleic, ref_nucleic, R_rmsd
print("Finished RMSD calculation.")

def plot_equivalent_systems(rmsd_data, rep):
    for system in ['6q2b', '5h3r']:
        equivalent_system = '5h3r' if system == '6q2b' else '6q2b'
        for condition in conditions[system]:
            if 'm' in condition:
                number = int(condition[1:])
                equivalent_number = number + 10 if system == '6q2b' else number - 10
                equivalent_condition = f'm{equivalent_number}'
            else:
                equivalent_condition = condition

            # Avoid creating duplicated graphs
            if system > equivalent_system:
                continue

            fig_path = f'rmsd_plot_{system}_{condition}_vs_{equivalent_system}_{equivalent_condition}_rep{rep}.png'
            if os.path.exists(fig_path):
                print(f"Skipping plot for {system}_{condition}_vs_{equivalent_system}_{equivalent_condition}_rep{rep} because it already exists.")
                continue

            try:
                rmsd_df_system = pd.DataFrame(rmsd_data[(system, condition, rep)].results.rmsd, columns=['Frame', 'Time (ns)', f'{system}_{condition}'])
                rmsd_df_equivalent = pd.DataFrame(rmsd_data[(equivalent_system, equivalent_condition, rep)].results.rmsd, columns=['Frame', 'Time (ns)', f'{equivalent_system}_{equivalent_condition}'])
                rmsd_df = pd.merge(rmsd_df_system, rmsd_df_equivalent, on=['Frame', 'Time (ns)'])

                # Check if 'Frame' column is empty or all values are the same
                if rmsd_df['Frame'].empty or rmsd_df['Frame'].nunique() == 1:
                    print(f"Skipping plot for {system}_{condition}_vs_{equivalent_system}_{equivalent_condition}_rep{rep} due to insufficient 'Frame' data.")
                    continue

                ax = rmsd_df.plot(x='Frame', y=[f'{system}_{condition}', f'{equivalent_system}_{equivalent_condition}'], kind='line')
                ax.set_ylabel(r'RMSD ($\AA$)')
                fig = ax.get_figure()
                fig.savefig(f'rmsd_plot_{system}_{condition}_vs_{equivalent_system}_{equivalent_condition}_rep{rep}.png')
                plt.close(fig)
            except Exception as e:
                print(f"Failed to plot {system}_{condition}_vs_{equivalent_system}_{equivalent_condition}_rep{rep}. Error: {str(e)}")

def plot_same_system(rmsd_data, rep):
    for system in ['6q2b', '5h3r']:
        fig_path = f'rmsd_plot_{system}_all_conditions_rep{rep}.png'
        if os.path.exists(fig_path):
            print(f"Skipping plot for {system}_all_conditions_rep{rep} because it already exists.")
            continue
        rmsd_df = pd.DataFrame()
        for condition in conditions[system]:
            rmsd_df_temp = pd.DataFrame(rmsd_data[(system, condition, rep)].results.rmsd, columns=['Frame', 'Time (ns)', f'{system}_{condition}'])
            if rmsd_df.empty:
                rmsd_df = rmsd_df_temp
            else:
                rmsd_df = pd.merge(rmsd_df, rmsd_df_temp, on=['Frame', 'Time (ns)'])
        ax = rmsd_df.plot(x='Frame', y=[f'{system}_{condition}' for condition in conditions[system]], kind='line')
        ax.set_ylabel(r'RMSD ($\AA$)')
        fig = ax.get_figure()
        fig.savefig(f'rmsd_plot_{system}_all_conditions_rep{rep}.png')
        plt.close(fig)

print("Starting to plot equivalent systems...")
for rep in range(1, 4):
    plot_equivalent_systems(rmsd_data, rep)
print("Finished plotting equivalent systems.")

print("Starting to plot same system, different conditions...")
for rep in range(1, 4):
    plot_same_system(rmsd_data, rep)
print("Finished plotting same system, different conditions.")

In [ ]:
#### Runs 2D-RMSD and HBs, one replicate at a time ####
#### PENDING

### Pairwise RMSD Calculation

# matrix_2d_rmsd = diffusionmap.DistanceMatrix(u, select='nucleic').run()
# fig, ax = plt.subplots()
# im = ax.imshow(matrix_2d_rmsd.dist_matrix, cmap='viridis')
# ax.set_xlabel('Frame')
# ax.set_ylabel('Frame')
# fig.colorbar(im, ax=ax, label=r'RMSD ($\AA$)')
# # plt.savefig(f'pairwise_rmsd_{protein_plotting}_{key}_{repl_no}.png')
# plt.savefig(f'pairwise_rmsd.png')
# plt.close(fig)

### Hydrogen Bonds Calculation



In [6]:
import os
import MDAnalysis as mda
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import nglview as nv
import ipywidgets as w
import ipykernel
from collections import defaultdict
from MDAnalysis.analysis import align, rms, diffusionmap
from MDAnalysisTests.datafiles import PSF, DCD
import MDAnalysis.transformations as trans
%matplotlib inline

### NOTE: Need to define how to deal with bad beginning of trajectories before continuing
### SOLVE: Propose that the first X frames (find a suitable number of ns, not a whole much % of total) were for equilibration and discard them --- Currently 75 frames, accounting for 1,5 ns out of 30 ns.

systems = ['6q2b', '5h3r']
conditions = {'6q2b': ['wild', 'm84', 'm63'], '5h3r': ['wild', 'm94', 'm73']}
mutations = {'6q2b': ['wild', 'k84r', 'r63t'], '5h3r': ['wild', 'r94k', 'r73t']}
rmsd_data = {}

def all_plots_exist(systems, conditions, reps):
    for system in systems:
        for condition in conditions[system]:
            for rep in reps:
                fig_path_same_system = f'rmsd_plot_{system}_all_conditions_rep{rep}.png'
                if not os.path.exists(fig_path_same_system):
                    return False

                equivalent_system = '5h3r' if system == '6q2b' else '6q2b'
                if 'm' in condition:
                    number = int(condition[1:])
                    equivalent_number = number + 10 if system == '6q2b' else number - 10
                    equivalent_condition = f'm{equivalent_number}'
                else:
                    equivalent_condition = condition

                if system <= equivalent_system:
                    fig_path_equivalent_system = f'rmsd_plot_{system}_{condition}_vs_{equivalent_system}_{equivalent_condition}_rep{rep}.png'
                    if not os.path.exists(fig_path_equivalent_system):
                        return False
    return True

reps = range(1, 4)
if all_plots_exist(systems, conditions, reps):
    print("All plots exist. Skipping RMSD calculation.")
else:
    print("Starting RMSD calculation for each system, condition, and replicate...")
    for system in systems:
        for condition in conditions[system]:
            mutation = mutations[system][conditions[system].index(condition)]
            for rep in range(1, 4):
                print(f"Processing {system}, {condition}, replicate {rep}...")
                dcd_file = f'{system}/analises_gerais/{mutation}/{system}_production_{condition}_reduced_rep{rep}.dcd'
                aligned_dcd_file = f'rmsfit_{system}_production_{condition}_reduced_rep{rep}.dcd'
                prmtop_file = f'{system}/analises_gerais/{mutation}/{system}_system_{condition}.top'

                if os.path.exists(aligned_dcd_file):
                    print(f"Using pre-aligned trajectory from {aligned_dcd_file}")
                    u = mda.Universe(prmtop_file, aligned_dcd_file, dt=200)
                    ref = mda.Universe(prmtop_file, dcd_file, dt=200)
                    protein = u.select_atoms('protein')
                    not_protein = u.select_atoms('not protein')
                    dna = u.select_atoms('nucleic')

                    transforms = [trans.unwrap(protein),
                                trans.center_in_box(protein, wrap=True),
                                trans.wrap(not_protein)]

                    u.trajectory.add_transformations(*transforms)

                    u_nucleic = u.select_atoms('nucleic')
                    ref_nucleic = ref.select_atoms('nucleic')

                    print("Calculating RMSD...")
                    R_rmsd = rms.RMSD(u_nucleic, ref_nucleic, select='nucleic', ref_frame=0)
                    R_rmsd.run(start=75, stop=1500, step=1)

                    rmsd_data[(system, condition, rep)] = R_rmsd

                    del u, protein, not_protein, dna, transforms, u_nucleic, ref_nucleic, R_rmsd

                else:
                    print(f"Aligning trajectory from {dcd_file}")
                    u = mda.Universe(prmtop_file, dcd_file, dt=200)
                    ref = mda.Universe(prmtop_file, dcd_file, dt=200)

                    protein = u.select_atoms('protein')
                    not_protein = u.select_atoms('not protein')
                    dna = u.select_atoms('nucleic')

                    transforms = [trans.unwrap(protein),
                                trans.center_in_box(protein, wrap=True),
                                trans.wrap(not_protein)]

                    u.trajectory.add_transformations(*transforms)

                    print("Running alignment...")
                    aligner = align.AlignTraj(u, ref, select='protein', in_memory=False).run(start=75, stop=1500, step=1)
                    u_nucleic = u.select_atoms('nucleic')
                    ref_nucleic = ref.select_atoms('nucleic')

                    print("Calculating RMSD...")
                    R_rmsd = rms.RMSD(u_nucleic, ref_nucleic, select='nucleic', ref_frame=0)
                    R_rmsd.run(start=75, stop=1500, step=1)

                    rmsd_data[(system, condition, rep)] = R_rmsd

                    del u, protein, not_protein, dna, transforms, aligner, u_nucleic, ref_nucleic, R_rmsd
print("Finished RMSD calculation.")

def plot_equivalent_systems(rmsd_data, rep):
    for system in ['6q2b', '5h3r']:
        equivalent_system = '5h3r' if system == '6q2b' else '6q2b'
        for condition in conditions[system]:
            if 'm' in condition:
                number = int(condition[1:])
                equivalent_number = number + 10 if system == '6q2b' else number - 10
                equivalent_condition = f'm{equivalent_number}'
            else:
                equivalent_condition = condition

            # Avoid creating duplicated graphs
            if system > equivalent_system:
                continue

            fig_path = f'rmsd_plot_{system}_{condition}_vs_{equivalent_system}_{equivalent_condition}_rep{rep}.png'
            if os.path.exists(fig_path):
                print(f"Skipping plot for {system}_{condition}_vs_{equivalent_system}_{equivalent_condition}_rep{rep} because it already exists.")
                continue

            try:
                rmsd_df_system = pd.DataFrame(rmsd_data[(system, condition, rep)].results.rmsd, columns=['Frame', 'Time (ns)', f'{system}_{condition}'])
                rmsd_df_equivalent = pd.DataFrame(rmsd_data[(equivalent_system, equivalent_condition, rep)].results.rmsd, columns=['Frame', 'Time (ns)', f'{equivalent_system}_{equivalent_condition}'])
                rmsd_df = pd.merge(rmsd_df_system, rmsd_df_equivalent, on=['Frame', 'Time (ns)'])

                # Check if 'Frame' column is empty or all values are the same
                if rmsd_df['Frame'].empty or rmsd_df['Frame'].nunique() == 1:
                    print(f"Skipping plot for {system}_{condition}_vs_{equivalent_system}_{equivalent_condition}_rep{rep} due to insufficient 'Frame' data.")
                    continue

                ax = rmsd_df.plot(x='Frame', y=[f'{system}_{condition}', f'{equivalent_system}_{equivalent_condition}'], kind='line')
                ax.set_ylabel(r'RMSD ($\AA$)')
                fig = ax.get_figure()
                fig.savefig(f'rmsd_plot_{system}_{condition}_vs_{equivalent_system}_{equivalent_condition}_rep{rep}.png')
                plt.close(fig)
            except Exception as e:
                print(f"Failed to plot {system}_{condition}_vs_{equivalent_system}_{equivalent_condition}_rep{rep}. Error: {str(e)}")

def plot_same_system(rmsd_data, rep):
    for system in ['6q2b', '5h3r']:
        fig_path = f'rmsd_plot_{system}_all_conditions_rep{rep}.png'
        if os.path.exists(fig_path):
            print(f"Skipping plot for {system}_all_conditions_rep{rep} because it already exists.")
            continue
        rmsd_df = pd.DataFrame()
        for condition in conditions[system]:
            rmsd_df_temp = pd.DataFrame(rmsd_data[(system, condition, rep)].results.rmsd, columns=['Frame', 'Time (ns)', f'{system}_{condition}'])
            if rmsd_df.empty:
                rmsd_df = rmsd_df_temp
            else:
                rmsd_df = pd.merge(rmsd_df, rmsd_df_temp, on=['Frame', 'Time (ns)'])
        ax = rmsd_df.plot(x='Frame', y=[f'{system}_{condition}' for condition in conditions[system]], kind='line')
        ax.set_ylabel(r'RMSD ($\AA$)')
        fig = ax.get_figure()
        fig.savefig(f'rmsd_plot_{system}_all_conditions_rep{rep}.png')
        plt.close(fig)

print("Starting to plot equivalent systems...")
for rep in range(1, 4):
    plot_equivalent_systems(rmsd_data, rep)
print("Finished plotting equivalent systems.")

print("Starting to plot same system, different conditions...")
for rep in range(1, 4):
    plot_same_system(rmsd_data, rep)
print("Finished plotting same system, different conditions.")


        # dcd_file = f'{system}/analises_gerais/{condition}/{system}_production_{condition}_reduced_rep1.dcd'
        # prmtop_file = f'{system}/analises_gerais/{condition}/{system}_system_{condition}.top'
        # u = mda.Universe(prmtop_file, dcd_file, dt=200)
        # u.transfer_to_memory(start=75, stop=1500, step=1)
        # ref = mda.Universe(prmtop_file, dcd_file, dt=200)
        # ref.transfer_to_memory(start=75, stop=1500, step=1)

        # protein = u.select_atoms('protein')
        # not_protein = u.select_atoms('not protein')
        # dna = u.select_atoms('nucleic')
        # ref_dna = mda.Merge(dna)

        # transforms = [trans.unwrap(protein),
        #             trans.center_in_box(protein, wrap=True),
        #             trans.wrap(not_protein)]

        # u.trajectory.add_transformations(*transforms)

        # # Aligned
        # aligner = align.AlignTraj(u, ref, select='protein', in_memory=True).run()
        # u_nucleic = u.select_atoms('nucleic')
        # ref_nucleic = ref.select_atoms('nucleic')

        # ### RMSD calculation
        # R_rmsd = rms.RMSD(u_nucleic,     # universe to align
        #             ref_nucleic,        # reference universe or atomgroup
        #             select='nucleic',   # group to superimpose and calculate RMSD
        #             ref_frame=0)        # frame index of the reference
        # R_rmsd.run()

        # rmsd_df = pd.DataFrame(R_rmsd.results.rmsd, columns=['Frame', 'Time (ns)', 'DNA'])

        ### Plot using pandas DataFrame method plot, can be improved surely
        # ax = rmsd_df.plot(x='Frame', y=['DNA'], kind='line')
        # ax.set_ylabel(r'RMSD ($\AA$)')
        # fig = ax.get_figure()
        # fig.savefig('rmsd_plot.png')
        # plt.close(fig)

        # rmsd_df[['Frame', 'DNA']].to_csv('RMSD_data.tsv', sep='\t', index=False, header=False)
        # rmsd_df[['Frame', 'Time (ns)', 'DNA']].to_csv('RMSD_data_timado.tsv', sep='\t', index=False, header=False)


### Pairwise RMSD Calculation

# matrix_2d_rmsd = diffusionmap.DistanceMatrix(u, select='nucleic').run()
# fig, ax = plt.subplots()
# im = ax.imshow(matrix_2d_rmsd.dist_matrix, cmap='viridis')
# ax.set_xlabel('Frame')
# ax.set_ylabel('Frame')
# fig.colorbar(im, ax=ax, label=r'RMSD ($\AA$)')
# # plt.savefig(f'pairwise_rmsd_{protein_plotting}_{key}_{repl_no}.png')
# plt.savefig(f'pairwise_rmsd.png')
# plt.close(fig)


### RMSF Calculation
### TODO: Gotta pass all 3 trajectories at once to generate a single RMSF plot
### Maybe we'll need to iterate in a different manner than in other analysis, but I think not
### Just run the RMSF universe generation and the rest when inside a key (wild, R63T, K84R...) for the first time

### TODO: Split the selection in half, only calculating for one of the monomers

# average = align.AverageStructure(u, u, select='protein and backbone',
#                                  ref_frame=0).run()
# ref_rmsf = average.results.universe

# aligner = align.AlignTraj(u, ref_rmsf,
#                           select='protein and backbone',
#                           in_memory=True).run()

# u_rmsf_backbone = u.select_atoms('protein and backbone')
# R_rmsf = rms.RMSF(u_rmsf_backbone).run()


# fig, ax = plt.subplots()
# ax.plot(u_rmsf_backbone.resids, R_rmsf.results.rmsf)
# ax.set_xlabel('Residue number')
# ax.set_ylabel('RMSF ($\AA$)')
# ax.legend()

### TODO: Write conditional to color the residues 63 and 84 if 6q2b and 73 and 94 if 5h3r
# # ax.axvspan(122, 159, zorder=0, alpha=0.2, color='orange', label='LID')
# # ax.axvspan(30, 59, zorder=0, alpha=0.2, color='green', label='NMP')

# fig.savefig(f'rmsf_plot_{protein_plotting}_{key}_{repl_no}.png')
# plt.close(fig)



All plots exist. Skipping RMSD calculation.
Finished RMSD calculation.
Starting to plot equivalent systems...
Skipping plot for 5h3r_wild_vs_6q2b_wild_rep1 because it already exists.
Skipping plot for 5h3r_m94_vs_6q2b_m84_rep1 because it already exists.
Skipping plot for 5h3r_m73_vs_6q2b_m63_rep1 because it already exists.
Skipping plot for 5h3r_wild_vs_6q2b_wild_rep2 because it already exists.
Skipping plot for 5h3r_m94_vs_6q2b_m84_rep2 because it already exists.
Skipping plot for 5h3r_m73_vs_6q2b_m63_rep2 because it already exists.
Skipping plot for 5h3r_wild_vs_6q2b_wild_rep3 because it already exists.
Skipping plot for 5h3r_m94_vs_6q2b_m84_rep3 because it already exists.
Skipping plot for 5h3r_m73_vs_6q2b_m63_rep3 because it already exists.
Finished plotting equivalent systems.
Starting to plot same system, different conditions...
Skipping plot for 6q2b_all_conditions_rep1 because it already exists.
Skipping plot for 5h3r_all_conditions_rep1 because it already exists.
Skipping plot

In [1]:
# Visualization, usually only works in native jupyter notebook server, not on VSCode
#outta_memory = nv.show_mdanalysis(u)
#outta_memory.add_representation('point', 'resname SOL')
#outta_memory.center()
#outta_memory

#view = nv.show_mdanalysis(u)
#view.add_representation('point', 'resname SOL')
#view.center()
#view

# w = universo.select_atoms('nucleic')
# view = nv.show_mdanalysis(ref_dna, gui=True)
# view

In [9]:

### Old code that'll likely be cannibalized ###

import os
import MDAnalysis as mda
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import nglview as nv
import ipywidgets as w
import ipykernel
from collections import defaultdict
from MDAnalysis.analysis import align, rms
import MDAnalysis.transformations as trans
from MDAnalysisTests.datafiles import PSF, DCD
%matplotlib inline

### NOTE: Need to define how to deal with bad beginning of trajectories before continuing
### SOLVE: Propose that the first X frames (find a suitable number of ns, not a whole much % of total) were for equilibration and discard them --- Currently 75 frames, accounting for 1,5 ns out of 30 ns.

systems = ['6q2b', '5h3r']
conditions = {'6q2b': ['wild', 'm84', 'm63'], '5h3r': ['wild', 'm94', 'm73']}
mutations = {'6q2b': ['wild', 'k84r', 'r63t'], '5h3r': ['wild', 'r94k', 'r73t']}
rmsd_data = {}

base_dir = '6q2b/analises_gerais/'

# Use defaultdict to automatically create a list when a new key is accessed
dcd_files = defaultdict(list)
top_files = defaultdict(list)

# Walk through the base directory and its subdirectories
for dirpath, dirnames, filenames in os.walk(base_dir):
    for file in filenames:
        # Get the parent directory of the file
        parent_dir = os.path.basename(dirpath)
        if file.endswith('.dcd'):
            dcd_files[parent_dir].append(os.path.join(dirpath, file))
        elif file.endswith('.top'):
            top_files[parent_dir].append(os.path.join(dirpath, file))


# Print the values in dcd_files and top_files
print("DCD Files:")
for key, value in dcd_files.items():
    print(f"{key}: {value}")

print("\nTOP Files:")
for key, value in top_files.items():
    print(f"{key}: {value}")

                    # Reference for
# 6q2b wild, namely using atom numbering in 6q2b_fixMET_noH2OandACY_pymolFixed.pdb
# --- @1-2429 (protein) and @2431-3491 (DNA)
# 6q2b R63T, using the numbering in 6q2b_R63T_mutantchimera.pqr
# --- @1-4999 (protein) and @5001-6651 (DNA)
# 6q2b K84R, using the numbering in 6q2b_K84R_mutantchimera.pqr
# --- @1-5024 (protein) and @5025-6676 (DNA)

# Load the trajectories
# for key in dcd_files.keys():
#     counter = 0
#     try:
#         for item in range(len(dcd_files[key])):
#             # print(dcd_files[key][item])
#             # print(top_files[key][0])
#             # print(key)
#             repl_no = dcd_files[key][item].split('_')[-1].split('.')[0]
#             protein = dcd_files[key][item].split('/')[0]
#             # print(protein)
#             # if protein == '6q2b':
#             if protein == '6q2b' and key == 'wild':
#                 if key == 'wild':
#                     try:
#                         # MDAnalysis, load the trajetory, possibly by chain ID and do: 2d_RMSD, RMSF, RoG and Clustering by RMSD
#                         universe = mda.Universe(top_files[key][0], dcd_files[key][item], dt = 0.002)
#                         reference_topology = mda.Universe(top_files[key][0])
#                         # Selecting the atoms for DNA and Protein
#                         dna = universe.select_atoms('nucleic')
#                         protein = universe.select_atoms('protein')
#                         # Reference Universe using DNA atoms
#                         reference_dna = mda.Merge(dna)
#                         print(f"reference_dna --- {reference_dna}")
#                         nv.show_mdanalysis(reference_dna)

#                     except Exception as e:
#                         print(f"Error loading trajectories for key {key} and item {item} --- Exception: {e}")

#                 elif key == 'R63T':
#                     try:
#                         traj = pt.load(dcd_files[key][item], top_files[key][0], mask='@1-4999')
#                         traj = pt.autoimage(traj)
#                         traj = pt.center(traj, '@5001-6651', mass=True)
#                         rmsd = pt.rmsd(traj, ref=0, mask='@1-4999')
#                         with open(f'6q2b_{key}_rmsd_for_{repl_no}.dat', 'w') as f:
#                             for value in rmsd:
#                                 f.write(f'{value}\n')
#                         print(f'6q2b_{key}_rmsd_for_{repl_no}.dat was written')
#                     except Exception as e:
#                         print(f"Error loading trajectories for {key}{item}: {e}")

#                 elif key == 'K84R':
#                     try:
#                         traj = pt.load(dcd_files[key][item], top_files[key][0], mask='@1-5024')
#                         traj = pt.autoimage(traj)
#                         traj = pt.center(traj, '@5025-6676', mass=True)
#                         rmsd = pt.rmsd(traj, ref=0, mask='@1-5024')
#                         with open(f'6q2b_{key}_rmsd_for_{repl_no}.dat', 'w') as f:
#                             for value in rmsd:
#                                 f.write(f'{value}\n')
#                         print(f'6q2b_{key}_rmsd_for_{repl_no}.dat was written')
#                     except Exception as e:
#                         print(f"Error loading trajectories for {key}{item}: {e}")

#                 else:
#                     print("Protein is adequate, but isn't wild, R63T or K84R. Skipping...")
#             else:
#                 print("Not the adequate protein. Skipping...")
#     except Exception as e:
#         print(f"Error loading trajectories for key: {key} item: {item}: with exception being {e}")


DCD Files:
wild: ['6q2b/analises_gerais/wild/6q2b_production_wild_reduced_rep1.dcd', '6q2b/analises_gerais/wild/6q2b_production_wild_reduced_rep3.dcd', '6q2b/analises_gerais/wild/6q2b_production_wild_reduced_rep2.dcd']
r63t: ['6q2b/analises_gerais/r63t/6q2b_production_m63_reduced_rep1.dcd', '6q2b/analises_gerais/r63t/6q2b_production_m63_reduced_rep2.dcd', '6q2b/analises_gerais/r63t/6q2b_production_m63_reduced_rep3.dcd']
k84r: ['6q2b/analises_gerais/k84r/6q2b_production_m84_reduced_rep1.dcd', '6q2b/analises_gerais/k84r/6q2b_production_m84_reduced_rep3.dcd', '6q2b/analises_gerais/k84r/6q2b_production_m84_reduced_rep2.dcd']

TOP Files:
wild: ['6q2b/analises_gerais/wild/6q2b_system_wild.top']
r63t: ['6q2b/analises_gerais/r63t/6q2b_system_m63.top']
k84r: ['6q2b/analises_gerais/k84r/6q2b_system_m84.top']
